In [29]:
import os

inputs = []
label = []

classes = []

dirts = 'ws_data'

for dirt in os.listdir(dirts):
    files_path = os.path.join(dirts, dirt)
    for file in os.listdir(files_path):
        text = ''
        file_path = os.path.join(files_path, file)
        with open(file_path, 'r', encoding='utf-8') as f:
            for l in f:
                text = text + l

        if text.strip() != '':
            inputs.append(text)
            label.append(dirt)

### Preprocessing

In [30]:
import pandas as pd

docs = pd.DataFrame()
docs['text'] = inputs
docs['class'] = label
classes = docs['class'].unique().tolist()

### Extracting Features

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text

stop_words = text.ENGLISH_STOP_WORDS.union()
count_vect = CountVectorizer(token_pattern="[^\W\d_]+")
train_data_features = count_vect.fit_transform(docs['text'])

def preprocess_file(file_path):
    text = ''
    with open(file_path, 'r', errors='ignore') as f:
        for l in f:
            text = l + ' '
    return count_vect.transform([text])

def preprocess(input):
    return count_vect.transform(input)

### Using Multinomial Naive Bayes Model from sklearn

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

docs['class'] = docs['class'].replace(docs['class'].unique().tolist(), range(0, len(docs['class'].unique())))
X = train_data_features.toarray()
Y = docs['class'].tolist()

x, _x, y, _y = train_test_split(X, Y, test_size=0.01, random_state=42)

clf = MultinomialNB()
_ = clf.fit(X, Y)

### Testing the Model

In [33]:
from sklearn.metrics import confusion_matrix
import numpy as np

yhat = clf.predict(_x)
matrix = confusion_matrix(_y, yhat)
accuracy = (matrix.diagonal() / matrix.sum(axis=1)) * 100

acc_table = pd.DataFrame()
acc_table['category'] = classes
acc_table['accuracy (%)'] = accuracy.round(1)

print(acc_table, '\n\nTotal Accuracy: ', (np.mean(yhat == _y)).round(1) * 100, '%')

               category  accuracy (%)
0         Art & Science         100.0
1               Finance         100.0
2  Goverment & Politics          80.0
3                Health         100.0
4  Science & Technology         100.0 

Total Accuracy:  100.0 %


### Manual Test

In [34]:
docs_text = ['Bacteria. These one-cell organisms are responsible for illnesses such as strep throat, urinary tract infections and tuberculosis.']

p_docs_text = preprocess(docs_text)
index = clf.predict(p_docs_text)[0]
print(index)

0


In [35]:
import pickle

pickle.dump(count_vect, open("preprocess.pkl","ab"))
pickle.dump(clf, open("model.pkl","ab"))

In [40]:
clf.predict(preprocess_file('../sample/Law Dictionary.pdf'))

FileNotFoundError: [Errno 2] No such file or directory: '../sample/Law Dictionary.pdf'